# Description

The purpose of the follow is to give us a model for price movements based on mean reversion. We do the following:

* Decompose the time series into trend, seasonal, and residual components using the STL decomposition method.

* Calculate the probabilities of the price being above or below the trend using a logistic regression model.

* Identify periods of high probability of the price being above or below the trend and attaching markers to the time series to indicate these periods.

* Identifying durations of high probability periods and calculating the extremes within each duration. The script will identify how long each duration lasted, and the maximum or minimum price movement within each duration.

* Fitting the distribution of the percentage deviations of the prices from the trend and identifying the probabilities of future price movements using a Poisson distribution.

* Creating various plots to visualize the results, including time series plots with trend, seasonal components, and probability markers, histograms of the probabilities, bar charts of the probability over time, and a bar chart of the probability density function of the percentage deviation of prices from the trend.



In [2]:
# Note: You can run this from the command line with scripts/mean_regressions.py
import plotly.io as pio
from plotly.offline import init_notebook_mode
from IPython.display import Image, display
import yfinance as yf
import pandas as pd
from machine_learning_finance import get_coin_data_frames, attach_markers, generate_probability, \
    calc_durations_with_extremes, prob_chart, plot_full_analysis, calculate_and_graph_duration_probabilities, \
    calculate_and_graph_price_probabilities, plot_full_analysis

init_notebook_mode(connected=True)
windows = [300, 600, 900, 1500]
window = 300
coin_base = False
ku_coin = False
use_stocks = True
symbol = "AAPL"
date_str = pd.Timestamp.now().strftime("%Y-%m-%d")
start_date = pd.to_datetime(date_str)

if not use_stocks:
    df_raw = get_coin_data_frames(window, symbol)
    df_raw = df_raw.set_index("Date")
    df_raw = df_raw.sort_index()
else:
    tickerObj = yf.download(tickers=symbol, interval="1d")
    df_raw = pd.DataFrame(tickerObj)
    df_raw = df_raw.tail(window)
    
trend, prob_above_trend, prob_below_trend, volatility, model = generate_probability(df_raw)

df_raw = attach_markers(df_raw, trend, prob_above_trend)
df_durations = calc_durations_with_extremes(df_raw)
plot_full_analysis(df_raw, trend, prob_above_trend, prob_below_trend, model, df_durations)
prob_chart(df_raw, prob_above_trend)


calculate_and_graph_duration_probabilities(start_date, df_raw, df_durations)

percent_diff_from_trend = ((df_raw["Close"] - trend) / trend) * 100
calculate_and_graph_price_probabilities(percent_diff_from_trend)



[*********************100%***********************]  1 of 1 completed
show


Last duration: 18
total: 0.9999944030151059


Current price diff: 16.111985288920526
